In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import requests
import os
from urllib.parse import urlparse

# Set up Chrome options
chrome_options = Options()
download_dir = "/Users/lisaraj/1.MASTER_DS/Clinic/2025-winter-data-and-democracy/notebooks/downloaded_pdfs"
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
})

# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Navigate to the website
url = "https://history.house.gov/Institution/Election-Statistics/Election-Statistics/"
driver.get(url)

# Find all the table rows
rows = driver.find_elements(By.CSS_SELECTOR, "table tr")

# Extract and print the href contents
pdf_links = []
for row in rows[1:]:  # Skip the header row
    cells = row.find_elements(By.TAG_NAME, "td")
    for cell in cells:
        link = cell.find_element(By.TAG_NAME, "a")
        href = link.get_attribute("href")
        print(href)
        pdf_links.append(href)

# Close the browser
driver.quit()

for link in pdf_links:
    parsed_url = urlparse(link)
    pdf_filename = os.path.basename(parsed_url.path)
    
    # Handle cases where pdf_filename is empty
    if not pdf_filename:
        pdf_filename = 'unnamed_file.pdf'
    
    filename = os.path.join("election_pdfs", pdf_filename)
    
    # Ensure the filename is unique
    counter = 1
    while os.path.exists(filename):
        name, ext = os.path.splitext(pdf_filename)
        filename = os.path.join("election_pdfs", f"{name}_{counter}{ext}")
        counter += 1
    
    response = requests.get(link)
    
    if response.status_code == 200:
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"Downloaded: {filename}")
    else:
        print(f"Failed to download: {link}")

In [ ]:
# Create the target directory if it doesn't exist
    output_dir = "election_pdfs"
    os.makedirs(output_dir, exist_ok=True)